In [2]:
import pyspark

In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/15 21:36:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.3.2
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.2
Branch HEAD
Compiled by user liangchi on 2023-02-10T19:57:40Z
Revision 5103e00c4ce5fcc4264ca9c4df12295d42557af6
Url https://github.com/apache/spark
Type --help for more information.


In [4]:
!pwd

/home/ccppyos/notebooks


In [8]:
df = spark.read.parquet('/home/ccppyos/notebooks/data/yellow_tripdata_2024-10.parquet')

In [9]:
df.repartition(4).write.parquet('data/yellow-2024-10',mode='overwrite')

In [15]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [35]:
import pyspark.sql.functions as  F


In [36]:
df.filter(F.to_date(df.tpep_pickup_datetime,"yyyy-MM-dd") == F.lit('2024-10-15')).count()

128893

In [37]:
##
df_extra = df.withColumn('DiffInSeconds', (col('tpep_dropoff_datetime') - col('tpep_pickup_datetime')).cast("long")/3600)

In [39]:
df_extra.select(F.max(F.col("DiffInSeconds"))).show()

+------------------+
|max(DiffInSeconds)|
+------------------+
|162.61777777777777|
+------------------+



In [29]:
df.head(1)

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 0, 30, 44), tpep_dropoff_datetime=datetime.datetime(2024, 10, 1, 0, 48, 26), passenger_count=1, trip_distance=3.0, RatecodeID=1, store_and_fwd_flag='N', PULocationID=162, DOLocationID=246, payment_type=1, fare_amount=18.4, extra=1.0, mta_tax=0.5, tip_amount=1.5, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=24.9, congestion_surcharge=2.5, Airport_fee=0.0)]

In [24]:
df.registerTempTable('yellow')

/home/ccppyos/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [25]:
df_yellow = spark.sql("""
SELECT
    DATE(tpep_pickup_datetime) as time,
    COUNT(1) as number_records
FROM
    yellow
WHERE
    DATE(tpep_pickup_datetime) = '2024-10-15'
""")

In [45]:
df_zones = spark.read.parquet('zones/')

In [48]:
df_zones.registerTempTable('zones')

In [46]:
df_zones.head(1)

[Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR')]

In [49]:
spark.sql("""
SELECT
    z.Zone,
    count(*) as rides
FROM yellow y
INNER JOIN zones z ON y.PULocationID = z.LocationID
GROUP BY 1
ORDER BY 2 
LIMIT 1
""").show()

+--------------------+-----+
|                Zone|rides|
+--------------------+-----+
|Governor's Island...|    1|
+--------------------+-----+



In [54]:
df_result = (df.join(df_zones, df.PULocationID == df_zones.LocationID, "inner")
            .groupBy("Zone")
            .agg(F.count("*").alias("rides"))
            .orderBy(F.col("rides"))
            .limit(1))


In [55]:
df_result.show()

+--------------------+-----+
|                Zone|rides|
+--------------------+-----+
|Governor's Island...|    1|
+--------------------+-----+



In [56]:
!pwd

/home/ccppyos/notebooks
